## Deep learning quantum DD compiler

## Methodology

1. **Model training.** The deep learning model was trained on 1,000 random 5-qubit circuits evaluated by IBMQ Burlington to predict the difference in trace distance between two equivalent random circuits. These supremacy-style random circuits had many gaps due to 2-qubit gates compensating for partial connectivity after compiling to maximum optimization; we filled the gaps completely randomly with sequences of U3 gates equivalent to the identity. All 1,000 circuits were run on IBM Q Burlington on March 15-16.

2. **Compiling.** We ran the trained model on a new set of 700 random circuits. For each random circuit (compiled to maximum optimization for IBM Q Burlington's error rates on March 18), we generated 1,000 randomly padded circuits. The deep learning model was then used to predict the relative noise between pairs of circuits in 50 tournaments of 20 competitors each, selecting the circuit expected to be the least noisy as the "compiled" circuit.

3. **Testing, part one.** The compiled circuits were run on IBM Q Burlington and IBM Q London, which have the same architecture. On March 23, the DD-compiled and IBM-compiled circuits were evaluated on Burlington and London. Keep in mind that the IBM-compiled circuits were compiled according to the Burlington error rates on March 18. Nevertheless, on Burlington, we found that DD-compiled circuits improve trace distance by 10% (7% to 14%, 0.95 CI) per circuit compared to IBM-compiled. On London, DD-compiled circuits make trace _worse_ by 13% (9% to 16%, 0.95 CI) compared to the circuits that were compiled according to Burlington error rates! This suggests that the DD-compiler may have learned non-Markovian noise unique to Burlington.

3. **Testing, part two.** The compiled circuits were run on IBM Q Burlington, Essex, London, Ourense, and Yorktown on March 27. Of these, only Yorktown has a different architecture. We expect that IBM Q Burlington will have departed somewhat compared to its original state reflected by the training set. Indeed, we found that DD-compiled improves trace distance by 7% (3% to 11%, 0.95 CI) per circuit compared to IBM-compiled; this is slightly (insignificantly) less improvement compared to the trials closer to the training set creation date. DD compiling _worsened_ performance on all other architectures compared to the IBM compiler: Essex by 20% (15% to 24%), London by 13% (9% to 16%), Ourense by 21% (15% to 27%), and Yorktown by 35% (28% to 43%). Perhaps unsurprisingly, the computer with the most different architecture was most worsened by the DD compiler. However, we also see that the compiler performance has worsened for IBM Q Burlington while remaining identical for IBM Q London!

In [1]:
import numpy as np
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats
import glob

Testing, part one:

In [33]:
directories = ['test_noise_5_only_burlington', 'test_noise_5_only_london']
for dr in directories:
    files = [glob.glob(dr + '/run1__compiled*'), glob.glob(dr + '/run1__identity*')]
    data = []
    for i in range(len(files)):
        d = []
        for f in sorted(files[i]):
            d.append(np.load(f))
        data.append(np.concatenate(d))
    diff1 = (data[0] - data[1])/data[1][:len(data[0])]
    diff2 = data[0] - data[1][:len(data[0])]
    print('Computer:', dr[dr.rfind('_')+1:])
    print('Mean % change in trace distance per circuit', bs.bootstrap(diff1, stat_func=bs_stats.mean))
    print('Mean absolute change in trace distance per circuit', bs.bootstrap(diff2, stat_func=bs_stats.mean))
    print()

Computer: burlington
Mean % change in trace distance per circuit -0.10446117455381426    (-0.14204148578167242, -0.06753752819691838)
Mean absolute change in trace distance per circuit -0.21104914285714285    (-0.25157852857142854, -0.16986549285714286)

Computer: london
Mean % change in trace distance per circuit 0.12609818342449383    (0.08647478102822229, 0.1655817400116898)
Mean absolute change in trace distance per circuit 0.020664285714285715    (-0.023902364285714274, 0.06349907857142856)



Testing, part two:

In [34]:
directories = ['test_noise_5_only_burlington', 'test_noise_5_only_essex', 'test_noise_5_only_london', 'test_noise_5_only_ourense', 'test_noise_5_only_yorktown']
for dr in directories:
    files = [glob.glob(dr + '/run2__compiled*'), glob.glob(dr + '/run2__identity*')]
    data = []
    for i in range(len(files)):
        d = []
        for f in sorted(files[i]):
            d.append(np.load(f))
        data.append(np.concatenate(d))
    diff1 = (data[0] - data[1])/data[1][:len(data[0])]
    diff2 = data[0] - data[1][:len(data[0])]
    print('Computer:', dr[dr.rfind('_')+1:])
    print('Mean % change in trace distance per circuit', bs.bootstrap(diff1, stat_func=bs_stats.mean))
    print('Mean absolute change in trace distance per circuit', bs.bootstrap(diff2, stat_func=bs_stats.mean))
    print()

Computer: burlington
Mean % change in trace distance per circuit -0.0696378753610124    (-0.11173340317778374, -0.02959570539435878)
Mean absolute change in trace distance per circuit -0.1736697142857143    (-0.2153510071428571, -0.13269476428571425)

Computer: essex
Mean % change in trace distance per circuit 0.19589913288687255    (0.14753075490251477, 0.24294665336582832)
Mean absolute change in trace distance per circuit 0.01928228571428572    (-0.007836778571428567, 0.04804202857142858)

Computer: london
Mean % change in trace distance per circuit 0.1273035426112978    (0.09080987122261508, 0.16361537649353125)
Mean absolute change in trace distance per circuit 0.04318085714285714    (0.000733821428571424, 0.08702069999999999)

Computer: ourense
Mean % change in trace distance per circuit 0.21072034134692783    (0.14413825237529293, 0.27403844634662367)
Mean absolute change in trace distance per circuit -0.024993428571428576    (-0.05811946428571428, 0.009027142857142845)

Compute